In [ ]:
!pip install sasl
!pip install thrift
!pip install thrift-sasl
!pip install Pyhive
!pip install tabulate

     |████████████████████████████████| 51kB 4.8MB/s eta 0:00:011
  Created wheel for sasl: filename=sasl-0.3.1-cp37-cp37m-linux_x86_64.whl size=232750 sha256=fad604717cbd8f0b9f16ec3f8fda054430d1c0f91ff9d6952dc13570e72268be
  Stored in directory: /root/.cache/pip/wheels/a3/2e/2f/d341ce73b59f464dd4c03e2b833712c0392a2bed0b7502a5bb
Successfully built sasl
     |████████████████████████████████| 61kB 5.2MB/s eta 0:00:011
  Created wheel for thrift: filename=thrift-0.15.0-cp37-cp37m-linux_x86_64.whl size=411333 sha256=b2ad8b9a0f68635d91905d3d1c9f137c0af4d8746b2987d264c4a24fc5322501
  Stored in directory: /root/.cache/pip/wheels/ed/98/a6/f324d326f5ebc20cf4aa06f0a1cffc29f0c31ed34830db24be
Successfully built thrift
  Created wheel for pure-sasl: filename=pure_sasl-0.6.2-cp37-none-any.whl size=11429 sha256=f60a3ec0cb891ef597e348cea7478974944ba477d8e66b40d02c1e25b65139f3
  Stored in directory: /root/.cache/pip/wheels/8d/34/73/6ed9e69a7ffd204e8d71bf422f9fa028f551d0dac431dc1082
Successfully built 

In [ ]:
from pyhive import hive
from tabulate import tabulate
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
hive_connection = hive.Connection(host='localhost',port=10000,username='saloni-cluster-m',password='1715895659092275074',database='default',auth='CUSTOM')


In [ ]:
hive_cursor = hive_connection.cursor()
hive_cursor.execute('SELECT PostId,DisplayName,sum(PostScore) as PostScore from salstackexchangeview GROUP BY PostId, DisplayName ORDER BY PostScore DESC LIMIT 10')

In [ ]:
result = hive_cursor.fetchall()

In [ ]:
result

[(11227809, '', 25903),
 (927358, '', 23303),
 (2003505, '', 18475),
 (292357, 'J. Pablo Fern&#225;ndez', 12812),
 (231767, 'Alex. S.', 11528),
 (477816, 'Oli', 10902),
 (348170, 'ceretullis', 10062),
 (5767325, '', 9899),
 (6591213, '', 9764),
 (1642028, '', 9545)]

In [ ]:
postdf = pd.read_sql("SELECT DISTINCT(PostId), DisplayName, PostTitle, PostMainBody FROM salstackexchangeview WHERE PostId IN ('11227809','927358','2003505','292357','231767','477816','348170','5767325','6591213','1642028') ORDER BY PostId",hive_connection)


In [ ]:
postdf.head()

,postid,displayname,posttitle,postmainbody
0,231767,Alex. S.,What does the yield keyword do,What is the use of the yield keyword in Python...
1,292357,J. Pablo Fern&#225;ndez,What is the difference between git pull and ...,What are the differences between git pull and ...
2,348170,ceretullis,How do I undo git add before commit,I mistakenly added files to Git using the comm...
3,477816,Oli,What is the correct JSON content type,I ve been messing around with JSON for some ti...
4,927358,,How do I undo the most recent local commits in...,I accidentally committed the wrong files to Gi...


In [ ]:
postdf["postid"] = postdf["posttitle"] + postdf["postmainbody"]

In [ ]:
top10_stackusername = list(postdf["displayname"].unique())
top10_stackusername

['Alex. S.', 'J. Pablo Fern&#225;ndez', 'ceretullis', 'Oli', '']

In [ ]:
# Calculate sum() of TF-IDF and get top 10 words with highest TF-IDF and select only those columns
def calculate_stack_tf_idf(df):
   ''' We will remove stop words and use postid to calculate tf-idf to find top 10 words''' 
   stack_vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
   response = stack_vectorizer.fit_transform(postdf["postid"])
   stackdf_tfidf_sklearn = pd.DataFrame(response.toarray(),columns=stack_vectorizer.get_feature_names())
   total_tfidf = stackdf_tfidf_sklearn.sum(axis = 0)
   top10_list = total_tfidf.nlargest(10)
   top10_words = list(top10_list.index)
   stackdf_tfidf_sklearn[top10_words]
   return stackdf_tfidf_sklearn[top10_words]


In [ ]:
## We shall iterate over all top 10 users to calculate tf-idf
for user in top10_stackusername: 
   filtered_data = postdf[(postdf['displayname']==user)] 
   tfidf_stackdf = calculate_stack_tf_idf(filtered_data) 
   print("For Username ID TF/IDF table : "+user)
   tfidf_stackdf.insert(0, 'usernameid', user)
   display(tfidf_stackdf)

For Username ID TF/IDF table : Alex. S.


,usernameid,git,branch,array,commit,local,lt,rename,json,data,undo
0,Alex. S.,0.000000,0.000000,0.000000,0.000000,0.000000,0.074909,0.000000,0.000000,0.000000,0.000000
1,Alex. S.,0.623080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Alex. S.,0.435000,0.000000,0.000000,0.467047,0.000000,0.000000,0.000000,0.000000,0.000000,0.311365
3,Alex. S.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.569455,0.000000,0.000000
4,Alex. S.,0.244690,0.000000,0.000000,0.233526,0.350288,0.000000,0.000000,0.000000,0.116763,0.233526
5,Alex. S.,0.000000,0.000000,0.000000,0.000000,0.000000,0.120961,0.000000,0.000000,0.000000,0.000000
6,Alex. S.,0.201329,0.528394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Alex. S.,0.000000,0.000000,0.532823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Alex. S.,0.174699,0.500183,0.000000,0.000000,0.333456,0.000000,0.588388,0.000000,0.000000,0.000000
9,Alex. S.,0.000000,0.000000,0.224539,0.000000,0.000000,0.436545,0.000000,0.000000,0.449079,0.000000


For Username ID TF/IDF table : J. Pablo Fern&#225;ndez


,usernameid,git,branch,array,commit,local,lt,rename,json,data,undo
0,J. Pablo Fern&#225;ndez,0.000000,0.000000,0.000000,0.000000,0.000000,0.074909,0.000000,0.000000,0.000000,0.000000
1,J. Pablo Fern&#225;ndez,0.623080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,J. Pablo Fern&#225;ndez,0.435000,0.000000,0.000000,0.467047,0.000000,0.000000,0.000000,0.000000,0.000000,0.311365
3,J. Pablo Fern&#225;ndez,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.569455,0.000000,0.000000
4,J. Pablo Fern&#225;ndez,0.244690,0.000000,0.000000,0.233526,0.350288,0.000000,0.000000,0.000000,0.116763,0.233526
5,J. Pablo Fern&#225;ndez,0.000000,0.000000,0.000000,0.000000,0.000000,0.120961,0.000000,0.000000,0.000000,0.000000
6,J. Pablo Fern&#225;ndez,0.201329,0.528394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,J. Pablo Fern&#225;ndez,0.000000,0.000000,0.532823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,J. Pablo Fern&#225;ndez,0.174699,0.500183,0.000000,0.000000,0.333456,0.000000,0.588388,0.000000,0.000000,0.000000
9,J. Pablo Fern&#225;ndez,0.000000,0.000000,0.224539,0.000000,0.000000,0.436545,0.000000,0.000000,0.449079,0.000000


For Username ID TF/IDF table : ceretullis


,usernameid,git,branch,array,commit,local,lt,rename,json,data,undo
0,ceretullis,0.000000,0.000000,0.000000,0.000000,0.000000,0.074909,0.000000,0.000000,0.000000,0.000000
1,ceretullis,0.623080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,ceretullis,0.435000,0.000000,0.000000,0.467047,0.000000,0.000000,0.000000,0.000000,0.000000,0.311365
3,ceretullis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.569455,0.000000,0.000000
4,ceretullis,0.244690,0.000000,0.000000,0.233526,0.350288,0.000000,0.000000,0.000000,0.116763,0.233526
5,ceretullis,0.000000,0.000000,0.000000,0.000000,0.000000,0.120961,0.000000,0.000000,0.000000,0.000000
6,ceretullis,0.201329,0.528394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,ceretullis,0.000000,0.000000,0.532823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,ceretullis,0.174699,0.500183,0.000000,0.000000,0.333456,0.000000,0.588388,0.000000,0.000000,0.000000
9,ceretullis,0.000000,0.000000,0.224539,0.000000,0.000000,0.436545,0.000000,0.000000,0.449079,0.000000


For Username ID TF/IDF table : Oli


,usernameid,git,branch,array,commit,local,lt,rename,json,data,undo
0,Oli,0.000000,0.000000,0.000000,0.000000,0.000000,0.074909,0.000000,0.000000,0.000000,0.000000
1,Oli,0.623080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Oli,0.435000,0.000000,0.000000,0.467047,0.000000,0.000000,0.000000,0.000000,0.000000,0.311365
3,Oli,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.569455,0.000000,0.000000
4,Oli,0.244690,0.000000,0.000000,0.233526,0.350288,0.000000,0.000000,0.000000,0.116763,0.233526
5,Oli,0.000000,0.000000,0.000000,0.000000,0.000000,0.120961,0.000000,0.000000,0.000000,0.000000
6,Oli,0.201329,0.528394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Oli,0.000000,0.000000,0.532823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Oli,0.174699,0.500183,0.000000,0.000000,0.333456,0.000000,0.588388,0.000000,0.000000,0.000000
9,Oli,0.000000,0.000000,0.224539,0.000000,0.000000,0.436545,0.000000,0.000000,0.449079,0.000000


For Username ID TF/IDF table : 


,usernameid,git,branch,array,commit,local,lt,rename,json,data,undo
0,,0.000000,0.000000,0.000000,0.000000,0.000000,0.074909,0.000000,0.000000,0.000000,0.000000
1,,0.623080,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,,0.435000,0.000000,0.000000,0.467047,0.000000,0.000000,0.000000,0.000000,0.000000,0.311365
3,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.569455,0.000000,0.000000
4,,0.244690,0.000000,0.000000,0.233526,0.350288,0.000000,0.000000,0.000000,0.116763,0.233526
5,,0.000000,0.000000,0.000000,0.000000,0.000000,0.120961,0.000000,0.000000,0.000000,0.000000
6,,0.201329,0.528394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,,0.000000,0.000000,0.532823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,,0.174699,0.500183,0.000000,0.000000,0.333456,0.000000,0.588388,0.000000,0.000000,0.000000
9,,0.000000,0.000000,0.224539,0.000000,0.000000,0.436545,0.000000,0.000000,0.449079,0.000000
